## Merlin Setup

The goal of this notebook is to create a user-managed container that can be used in Vertex Workbench

### Setup variables, paths, and create artifact registry

In [3]:
VERSION=23.02 #nvcr.io/nvidia/merlin/merlin-pytorch:23.02
REPO_NAME="workbench"
REGION="us-central1"
PROJECT="wortz-project-352116" # TODO: update with your project_id
IMAGE_ID="pytorch"
MERLIN_IMAGE_NAME=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPO_NAME}/merlin-{IMAGE_ID}-{VERSION}"

MERLIN_CONTAINER="merlin_container"

!mkdir {MERLIN_CONTAINER}

!gcloud beta artifacts repositories create {REPO_NAME} \
    --repository-format=docker \
    --location=$REGION

mkdir: cannot create directory ‘merlin_container’: File exists
ERROR: (gcloud.beta.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [4]:
!gcloud config set project $PROJECT

Updated property [core/project].


#### Find your service account for you tenant project - usually project_id-compute@developer.gserviceaccount.com

In [5]:
# !gcloud projects add-iam-policy-binding hybrid-vertex --member=serviceAccount:xxxxxxxx-compute@developer.gserviceaccount.com --role=roles/artifactregistry.admin

In [6]:
!gcloud auth configure-docker us-central1-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
Docker configuration file updated.


### Create Docker image derived from Deeplearning containers

In [7]:
%%writefile {MERLIN_CONTAINER}/jupyter_notebook_config.py
c.NotebookApp.ip = '*'
c.NotebookApp.token = ''
c.NotebookApp.password = ''
c.NotebookApp.open_browser = False
c.NotebookApp.port = 8080
c.NotebookApp.terminado_settings = {'shell_command': ['/bin/bash']}
c.NotebookApp.allow_origin_pat = (
'(^https://8080-dot-[0-9]+-dot-devshell\.appspot\.com$)|'
'(^https://colab\.research\.google\.com$)|'
'((https?://)?[0-9a-z]+-dot-(?:us|asia|europe|northamerica|southamerica)-?[0-9a-z]+\.notebooks\.googleusercontent.com)')
c.NotebookApp.allow_remote_access = True
c.NotebookApp.disable_check_xsrf = False

Overwriting merlin_container/jupyter_notebook_config.py


In [53]:
%%writefile {MERLIN_CONTAINER}/Dockerfile
FROM nvcr.io/nvidia/merlin/merlin-pytorch:23.02
RUN echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt cloud-sdk main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list && curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key --keyring /usr/share/keyrings/cloud.google.gpg  add - && apt-get update -y && apt-get install google-cloud-sdk -y
EXPOSE 8080

# copy the dependencies file to the working directory
COPY merlin_container/requirements.txt .



RUN export GCSFUSE_REPO=gcsfuse-$(lsb_release -c -s) && \
    echo ${GCSFUSE_REPO} && \
    ( echo "deb http://packages.cloud.google.com/apt ${GCSFUSE_REPO} main" \
      | tee /etc/apt/sources.list.d/gcsfuse.list ) && \
    more /etc/apt/sources.list.d/gcsfuse.list && \
    ( curl https://packages.cloud.google.com/apt/doc/apt-key.gpg \
      | apt-key add - )

RUN apt update && apt -y install gcsfuse

RUN apt-get update
RUN apt-get install gcsfuse

# install dependencies
RUN apt-get install nodejs npm -y
RUN pip install -r requirements.txt
#RUN mkdir /root/.jupyter
             
COPY merlin_container/jupyter_notebook_config.py /root/.jupyter

ENV pwd=""
ENTRYPOINT exec jupyter-lab --ip=0.0.0.0 --port=8080 --no-browser --allow-root --ServerApp.allow_origin="*" --NotebookApp.token="$pwd" --NotebookApp.password="$pwd"

Overwriting merlin_container/Dockerfile


In [54]:
%%writefile {MERLIN_CONTAINER}/requirements.txt
fastapi
merlin-models
gsutil
gcsfs
matplotlib
google-cloud-aiplatform
jupyterlab-git

Overwriting merlin_container/requirements.txt


In [ ]:
!docker build . -f $MERLIN_CONTAINER/Dockerfile -t $MERLIN_IMAGE_NAME
!docker push $MERLIN_IMAGE_NAME

Sending build context to Docker daemon  2.772MB
Step 1/13 : FROM nvcr.io/nvidia/merlin/merlin-pytorch:23.02
 ---> d571f1f72064
Step 2/13 : RUN echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt cloud-sdk main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list && curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key --keyring /usr/share/keyrings/cloud.google.gpg  add - && apt-get update -y && apt-get install google-cloud-sdk -y
 ---> Using cache
 ---> 667d60766ef0
Step 3/13 : EXPOSE 8080
 ---> Using cache
 ---> 9fa1b490f7a5
Step 4/13 : COPY merlin_container/requirements.txt .
 ---> Using cache
 ---> c4cf7d2685ef
Step 5/13 : RUN export GCSFUSE_REPO=gcsfuse-$(lsb_release -c -s) &&     echo ${GCSFUSE_REPO} &&     ( echo "deb http://packages.cloud.google.com/apt ${GCSFUSE_REPO} main"       | tee /etc/apt/sources.list.d/gcsfuse.list ) &&     more /etc/apt/sources.list.d/gcsfuse.list &&     ( curl https://packages.cloud.google.c